In [ ]:
import os
import pathlib
import random
import sys

import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()
INSPECTION_PATH = pathlib.Path(os.getcwd()).parent / "country-backend"
sys.path.append(str(INSPECTION_PATH.absolute()))

In [ ]:
from country.model.predict import model
from country.model.predict import preprocess, prediction, predict_image

In [ ]:
from country.explainer.new_lime_ import explain_image, create_segments, generate_samples, generate_images, predict_images, weigh_segments, generate_visual_explanation
from PIL import Image


In [ ]:
imgg = preprocess(cv2.imread("C:\\Users\\Matze\\Documents\\University\\Work\\BeispielBilder\\B\\jerusalem.jpg"))
print(predict_image(imgg))
print(imgg.shape)
imgg = np.squeeze(imgg) #if np.squeeze() is off, then an error raises in generate_images (more info in the method around line 91)

In [ ]:
from skimage.segmentation import mark_boundaries
import time
seg= create_segments(img = imgg, seg_method = "slic", settings = {})
plt.imshow(mark_boundaries(imgg,seg))
print(len(seg))

In [ ]:
"""s = time.time()
image_explanation = explain_image(img = imgg, seg_method = "slic", seg_settings = {}, num_of_samples = 700, samples_p = 0.5,
                  model_ = model, threshold = 0.5, volume = 20, colour =  "green", transparency = 0.7)
print(time.time() - s)"""



s=time.time()
segment_mask = create_segments(img=imgg, seg_method="slic", settings={})
print("create_segments: ", time.time()-s)
print()
s=time.time()
samples_theo = generate_samples(segment_mask=segment_mask, num_of_samples=700, p=0.5)
print("generate_samples: ", time.time()-s)
print()
s=time.time()
samples_imgs = generate_images(image=imgg, segment_mask=segment_mask, samples=samples_theo)
print("generate_images: ", time.time()-s)
print()

s=time.time()
samples_imgs_predictions = predict_images(images=samples_imgs, model_=model)
print("predict_images: ", time.time()-s)
print()

s=time.time()
weighted_segments = weigh_segments(samples=samples_theo, predictions=samples_imgs_predictions)
print("weigh_segments: ", time.time()-s)
print()

s=time.time()
visual_explanation = generate_visual_explanation(weighted_segments=weighted_segments, segment_mask=segment_mask,
                                                  image=imgg, threshold=0.5, volume=20, colour="green", transparency= 0.7)
print("generate_visual_explanation: ", time.time()-s)




predict_images:  17.770128965377808

weigh_segments:  0.01299905776977539



KeyboardInterrupt: 

In [ ]:


f, axarr = plt.subplots(1,2, figsize = (20, 8))
axarr[0].axis("off")
axarr[1].axis("off")
#axarr[0].imshow(((visual_explanation + 1) *127.5 )/255)
axarr[0].imshow((((visual_explanation +1)*127.5)/255))
axarr[1].imshow(((imgg + 1) *127.5 )/255)

In [ ]:
# green tones have to differ more (use other model?)
#Notes:
# +1)*127.5) maybe unnecessary for other models (this occurs in lime (line 192) as well)